<a href="https://colab.research.google.com/github/gansarpamungkas/sentiment_analysis_pemilu_indonesia_2019/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [9]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project 1/dataset/preprocessing.csv')
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1815 entries, 0 to 1814
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          1815 non-null   int64 
 1   sentimen            1815 non-null   object
 2   tweet               1815 non-null   object
 3   tweet_no_stopwords  1815 non-null   object
 4   stem                1815 non-null   object
dtypes: int64(1), object(4)
memory usage: 71.0+ KB


None

## Model Building for tf/idf

In [10]:
!pip install nlp_id
!pip install scikit-learn

In [11]:
 from sklearn.feature_extraction.text import CountVectorizer
 import pandas as pd
 from nlp_id import StopWord

 stopword = StopWord()
 stop_words = stopword.get_stopword()
 print(stop_words)

['ada', 'adalagi', 'adalah', 'adanya', 'adapun', 'agak', 'agak-agak', 'agaknya', 'agar', 'aja', 'akan', 'akankah', 'akankan', 'akhir', 'akhir-akhirnya', 'akhirannya', 'akhiri', 'akhirinya', 'akhirnya', 'aku', 'akulah', 'akunya', 'akurat', 'akutu', 'ala', 'alamak', 'alhamdulillah', 'alhasil', 'amat', 'amat-amat', 'amatlah', 'anda', 'anda-anda', 'andai', 'andalah', 'antar', 'antara', 'antaranya', 'apa', 'apa-apanya', 'apaan', 'apabila', 'apaitu', 'apakah', 'apalagi', 'apanya', 'apasaja', 'apatah', 'apesnya', 'arah', 'artinya', 'asal', 'asalkan', 'asumsinya', 'asumsikan', 'atas', 'atas-atas', 'atas-batas', 'ataspun', 'atau', 'ataukah', 'ataupun', 'awal', 'awalnya', 'bagai', 'bagaikan', 'bagaimana', 'bagaimanakah', 'bagaimanapun', 'bagi', 'bagi-bagi', 'bagian', 'bagikan', 'bahasakan', 'bahasanya', 'bahkan', 'bahwa', 'bahwasanya', 'baik', 'bak', 'bakal', 'bakal-bakal', 'bakalan', 'balik', 'balikan', 'balikkan', 'banget', 'banget-banget', 'banyak', 'barangkali', 'baru', 'baru-baru', 'bawah',

In [12]:
dokumen = df['stem']

### LSTM

In [13]:
X = df['stem']
y = df['sentimen']
X


0       prabowo indonesia tidak harga bangsa asing ber...
1       batu langka tasbih jokowi hadiah habib luthfi ...
2       era jokowi ekonomi indonesia 01indonesiamaju j...
3       sumatera selatan asi games dampak ekonomi lang...
4       negara ngutang bangun infrastruktur pakai masy...
                              ...                        
1810    negarawan sejati sll bangga depan harga bangsa...
1811    1 ceramah damai indonesia 2 ekonomi baik putih...
1812    mari bangun bangsa dukung ekonomi negara jokow...
1813               bantu maju ekonomi bangsa jokowi yuk a
1814    jokowi ubah pandang ekonomi orang hubung indon...
Name: stem, Length: 1815, dtype: object

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [15]:
X = df['stem']
y = df['sentimen']

tfidf = TfidfVectorizer()
X = tfidf.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, stratify = y)

print('Shape of X:', X.shape)


Shape of X: (1815, 5952)


In [16]:
X_train

<1452x5952 sparse matrix of type '<class 'numpy.float64'>'
	with 20348 stored elements in Compressed Sparse Row format>

In [17]:
X_train = X_train.toarray()[:, :, None]
X_test = X_test.toarray()[:, :, None]

In [18]:
!pip install scikeras

In [19]:
import tensorflow as tf
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import GridSearchCV
import numpy as np

# Define a function for creating the LSTM model with a specified dropout_rate
def create_lstm_model(input_shape, units=50, optimizer='adam', dropout_rate=0.2):
    model = Sequential()
    model.add(LSTM(units, input_shape=input_shape))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Define the parameter grid for hyperparameter tuning without dropout_rate
param_grid = {
    'units': [50],
    'optimizer': ['adam']
}

best_score = 0
best_dropout_rate = 0.2

# Iterate through different dropout rates and find the best one
for dropout_rate in [0.2, 0.3, 0.4]:
    input_shape = (X_train.shape[1], X_train.shape[2])
    model = KerasClassifier(build_fn=create_lstm_model, input_shape=input_shape, units=100, dropout_rate=dropout_rate, optimizer='adam', epochs=10, batch_size=32, verbose=0)
    grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3)
    grid_result = grid.fit(X_train, y_train)

    print(f"Dropout Rate: {dropout_rate}, Accuracy: {grid_result.best_score_}")

    if grid_result.best_score_ > best_score:
        best_score = grid_result.best_score_
        best_dropout_rate = dropout_rate

# Print the best dropout_rate and hyperparameter tuning results
print("Best Dropout Rate: %f" % best_dropout_rate)
print("Best Accuracy: %f" % best_score)


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Dropout Rate: 0.2, Accuracy: 0.3285123966942149


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt
from plotly.offline import iplot
import seaborn as sns

In [ ]:
plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show();

In [ ]:
plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show();